In [1]:
import pandas as pd
import sqlite3 as sq
from sklearn.model_selection import train_test_split

conn = sq.connect("../paladins.db")

df = pd.read_sql_query("SELECT * FROM match", conn)

df["result"] = df["result"].apply(lambda x: 0 if x == 'L' else 1)

df = pd.get_dummies(df, columns=["champion"], dtype=int)

In [2]:


champion_cols = [col for col in df.columns if col.startswith("champion_")]
agg_dict = {col: 'sum' for col in champion_cols}
agg_dict.update({"result" : "first", "match_id" : "first"})

df = df.groupby(["match_id", "result"], as_index=False).agg(agg_dict).reset_index()

X = df[champion_cols]
y = df["result"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
import numpy as np
import matplotlib as plt
import tensorflow as tf
from keras import layers

model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_dim=X_train.shape[1]),
    layers.Dropout(0.3),                                            
    layers.Dense(32, activation='relu'),                            
    layers.Dense(1, activation='sigmoid')                            
])

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])


/Users/mateomembreno/Documents/Projects/Paladins-Ranker/env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
history = model.fit(X_train, y_train, 
                    epochs=10, 
                    batch_size=32, 
                    validation_split=0.2, 
                    verbose=1)

# Evaluate on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

# Predict probabilities for the test set
y_pred_proba = model.predict(X_test)
print(f"Predict accuracy: {y_pred_proba}")

val_loss = history.history["val_binary_crossentropy"]
plt.plot(np.arange(1,len(val_loss)+1),val_loss)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6523 - loss: 0.6300 - val_accuracy: 0.5440 - val_loss: 0.7048
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6558 - loss: 0.6151 - val_accuracy: 0.5660 - val_loss: 0.7058
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6415 - loss: 0.6197 - val_accuracy: 0.5472 - val_loss: 0.7102
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6684 - loss: 0.6138 - val_accuracy: 0.5535 - val_loss: 0.7109
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6748 - loss: 0.5931 - val_accuracy: 0.5629 - val_loss: 0.7176
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7107 - loss: 0.5707 - val_accuracy: 0.5723 - val_loss: 0.7251
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7086 - loss: 0.5685 - val_accuracy: 0.5189 - val_loss: 0.7306
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7129 - loss: 0.5690 - val_accuracy: 0.5660 - val_loss:

KeyError: 'val_binary_crossentropy'